In [4]:
!pip install langchain
!pip install pdfminer.six
!pip install InstructorEmbedding
!pip install -U sentence-transformers
!pip install --pre -U "weaviate-client==4.*"

In [5]:
from langchain.document_loaders import TextLoader, PDFMinerLoader, CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceInstructEmbeddings
from transformers import AutoModelForCausalLM
from langchain.chains import RetrievalQA
from google.colab import userdata

In [6]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
SOURCE_DIRECTORY = "source-docs/8-tsne.pdf"

In [8]:
import weaviate
from langchain.vectorstores.weaviate import Weaviate
from google.colab import userdata

# weaviate_url = "https://exactas-guru-cluster-1awnzsok.weaviate.network"
# auth_config = weaviate.AuthApiKey(api_key=userdata.get('WEAVIATE_API_KEY'))

client = weaviate.Client("http://localhost:8080")
vectorstore = Weaviate(client, "PodClip", "content")

In [9]:
print(f"Loading documents from {SOURCE_DIRECTORY}")
loader = PDFMinerLoader(SOURCE_DIRECTORY)
document = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(document)
print(f"Split into {len(texts)} chunks of text")

Loading documents from drive/MyDrive/rag-agent-source-docs/8-tsne.pdf
Split into 107 chunks of text


In [10]:
embeddings = HuggingFaceInstructEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
# import weaviate

# client = weaviate.connect_to_wcs(
#     cluster_url=weaviate_url,  # Replace with your WCS URL
#     auth_credentials=weaviate.AuthClientPassword(
#         username=userdata.get('WCS_USERNAME'),  # Your WCS username
#         password=userdata.get('WCS_PASSWORD')   # Your WCS password
#     )
# )

WeaviateGrpcUnavailable: ignored

In [11]:
db = Weaviate.from_documents(texts, embeddings, weaviate_url=weaviate_url, by_text=False)

AuthenticationFailedException: ignored

In [ ]:
llm = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", load_in4_bit=True)

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff")
# , retriever=db.as_retriever(search_kwargs={'k': 2})

In [ ]:
query = "why does t-sne use t-distribution?"

qa.run(query)